# Case Study: data preparation of loans data for self service BI

<b>REMARK:</b> For this case study we will be using the hortonbox (the version was 2.3.2 at the time of writing this chapter)

### Things to do on the hortonbox before the system works:

<p> <b>REMARK:</b> you can connect to the hortonbox via putty, this is usually easier than working on the VM itself</p>
<p>yum -y install python-pip</p>
<p>pip install git+https://github.com/DavyCielen/pywebhdfs.git --upgrade</p>
<p> <b>REMARK:</b> this custom repository is a fork of pywebhdfs. pywebhdfs was broken at the time of writing. A reqest to fix it has been issued but until the main repo works properly again, use the custom one we created</p>
<p>pip install pandas</p>

<p><b>REMARK:</b> all the following <b>code</b> will need to be run <b>on the hortonbox</b> itself</p>
<p>To do this, open a putty session and issue the "pyspark" command, this will open a python interpreter</p>
<p>If all went well before, this interpreter should also have the pandas and pywebhdfs (or adapted pywebhdfs) libraries </p>
<p>All the following code should be run in this pyspark session (you can just copy paste it piece by piece)</p>
<p>There are other options like copying the code to a .py file and running it there, another option is the zeppelin notebook</p>
<p>Zeppelin is installed on hortonbox but was not mature enough to rely on at the time of writing.</p>

## Part 1 of data science process : research goal

<b>Main learning goal:</b> Using big data technologies to preparation data for <b>self-service BI</b>. In this case study we will not get into model building but instead spend some more time on the data preparation in order to allow other people to explore it and eventually create reports using the prepared data. The self-service BI concept allows end-users to find their own insights and is often applied when the company is very data driven but not enough data scientists are present to provide the insights. Chapter 9 will also feature a case study with some more focus on the application of the dashboard layer for self service BI. 

## Part 2 of data process : data retrieval

### Download the data

In [6]:
import requests
import zipfile
import StringIO 

In [ ]:
source = requests.get("https://resources.lendingclub.com/LoanStats3d.csv.zip", verify=False) #A
stringio = StringIO.StringIO(source.content) #B
unzipped = zipfile.ZipFile(stringio) #C

In [ ]:
#A Download the data from the lending club. This https so it should verify but we won't bother (verify=False) 
#B creates a virtual file.
#C Unzip the data

### Move data to hadoop

In [ ]:
import pandas as pd 
from pywebhdfs.webhdfs import PyWebHdfsClient

In [4]:
subselection_csv = pd.read_csv(unzipped.open('LoanStats3d.csv'),skiprows=1,skipfooter=2,engine='python') #A
stored_csv = subselection_csv.to_csv('./stored_csv.csv') #B

In [ ]:
hdfs = PyWebHdfsClient(user_name="hdfs",port=50070,host="sandbox")#C
hdfs.make_dir('chapter5')#D
with open('./stored_csv.csv') as file_data: #E
        hdfs.create_file('chapter5/LoanStats3d.csv',file_data, overwrite=True)#F

In [ ]:
#A Do some preliminary data cleaning using Pandas: remove top row and bottom 2 rows because they are useless to us. simply opening the original file will show you this.
#B Store it locally because we need to transfer it to the hadoop file system
#C Connect to hadoop sandbox
#D Create a folder called "chapter5" on hadoop filesystem
#E Open the locally stored csv 
#F Create the .csv file on hadoop filesystem

In [ ]:
print hdfs.get_file_dir_status('chapter5/LoanStats3d.csv')#A

In [ ]:
#A print the file status

In [ ]:
#hdfs.delete_file_dir('chapter5/LoanStats3d.csv',recursive=True)#A

In [ ]:
#A code in case you want to delete a file

## Part 3 of data process : data preparation

### Apache Spark data cleaning

In [ ]:
from pyspark import SparkContext #A
from pyspark.sql import HiveContext #B
#sc = SparkContext()#C
sqlContext = HiveContext(sc)#D
data = sc.textFile("/chapter5/LoanStats3d.csv") #E
parts = data.map(lambda r:r.split(',')) #F
firstline = parts.first() #G
datalines = parts.filter(lambda x:x != firstline)#H
def cleans(row): #I
        row[7] = str(float(row[7][:-1])/100)#J
        return [s.encode('utf8').replace(r"_"," ").lower() for s in row]#K

In [ ]:
datalines = datalines.map(lambda x: cleans(x))#L

In [ ]:
#A import Spark context --> not necessary when working directly in pyspark!
#B import Hive context
#C In the pyspark session the sparkcontext is automatically present. In  other cases (zeppelin notebook) you will need to 
# create this explicitly
#D Create the hivecontext 
#E load in the dataset from the hadoop directory
#F split the dataset with the komma (,) delimiter. This is the end of line delimiter for this file
#G grab the first line
#H grab all the lines but the first line. because the first line is just variable names
#I the cleaning function will use the power of Spark to clean the data. The input of this function will be a line of data.
#J column 8 (index = 7) has % formatted numbers. We don't need that '%' sign.
#K Encode everything in utf8, replace underscores with spaces and lowercase everything.
#L excecute the data cleaning line by line

### Saving data in hive

In [ ]:
from pyspark.sql.types import * #A
fields = [StructField(field_name,StringType(),True) for field_name in firstline] #B
schema = StructType(fields) #C
schemaLoans = sqlContext.createDataFrame(datalines, schema) #D
schemaLoans.registerTempTable("loans") #E

sqlContext.sql("drop table if exists LoansByTitle") #F
sql = '''create table LoansByTitle stored as parquet as select title, count(1) as number from loans group by title order by number desc'''#F
sqlContext.sql(sql)#F

sqlContext.sql('drop table if exists raw') #G
sql = '''create table raw stored as parquet as select title, emp_title,grade,home_ownership,int_rate,recoveries,collection_recovery_fee,loan_amnt,term from loans'''#G
sqlContext.sql(sql)#G

In [ ]:
#A Import sql data types 
#B Create metadata: the Spark SQL StructField function represents a field in a StructType. 
# StructField object is comprised of three fields: name (a string), dataType (a DataType) and 'nullable' (a boolean). 
#The field of name is the name of a StructField. 
#The field of dataType specifies the data type of a StructField.
#The field of nullable specifies if values of a StructField can contain None values.
#C StructType function creates the data schema. A StructType object requires a list of StructFields as imput.
#D Create a dataframe from the data (datalines) and the dataschema (schema)
#E Register it as a table called loans 
#F Drop table (in case it already exists), summarize and store in hive. LoansByTitle represents the sum of loans by job title
#G Drop table (in case it already exists) and store a subset of the raw data in Hive

## Part 4 of data process : data exploration & part 6: presentation to end-user

The data exploration part of this case study will be done in Qlick. There is no real modelling phase in this case study. Because Qlick allow you to build dashboard and reports, part 4 and 6 form an overlap. 